In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import keras as kk
from keras.layers import Dense
import logging

Using TensorFlow backend.
/home/hisense/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hisense/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hisense/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hisense/anaconda3/lib/python3.7/site-packages/t

In [2]:
import pickle
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas

In [3]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
data = pd.read_csv('../data2/data_12xx.txt', sep='\t')
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,qid,qid_enc,...,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,uid,uid_enc,wk
0,3865,-0.122170,3.0,2,22,0.021705,0.0,NaN,Q2166419046,154133,...,0,0,2113,190,261,927,1,M401693808,1508098,1
1,3862,-0.015598,3.0,2,15,-0.182562,0.0,NaN,Q604029601,458113,...,0,0,1519,229,0,506,1,M2317670257,657985,5
2,3867,0.000000,2.0,2,4,0.070942,0.0,32.0,Q2443223942,190601,...,0,0,551,226,188,815,1,M3544409350,1272353,3
3,3861,-0.302475,4.0,1,20,-0.191518,0.0,3.0,Q795459266,483270,...,0,0,1519,229,0,506,1,M2818659842,909154,4
4,3862,-0.095008,3.0,0,8,-0.176807,1.0,NaN,Q110462128,13599,...,0,0,2161,31,396,1438,1,M848334644,1856019,5


In [17]:
feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [18]:
train_label=2593669

In [19]:
data = data.fillna(0)
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,qid,qid_enc,...,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,uid,uid_enc,wk
0,3865,-0.122170,3.0,2,22,0.021705,0.0,0.0,Q2166419046,154133,...,0,0,2113,190,261,927,1,M401693808,1508098,1
1,3862,-0.015598,3.0,2,15,-0.182562,0.0,0.0,Q604029601,458113,...,0,0,1519,229,0,506,1,M2317670257,657985,5
2,3867,0.000000,2.0,2,4,0.070942,0.0,32.0,Q2443223942,190601,...,0,0,551,226,188,815,1,M3544409350,1272353,3
3,3861,-0.302475,4.0,1,20,-0.191518,0.0,3.0,Q795459266,483270,...,0,0,1519,229,0,506,1,M2818659842,909154,4
4,3862,-0.095008,3.0,0,8,-0.176807,1.0,0.0,Q110462128,13599,...,0,0,2161,31,396,1438,1,M848334644,1856019,5


In [20]:
dd_cols=['uf_c1','uf_c2','uf_c3','uf_c4','uf_c5','uid_enc','qid_enc']
for cols in dd_cols:
    t = data[cols]
    max1=t.max()
    min1=t.min()
    tqdm.pandas(desc="{cols}...")
    data[cols]=data[cols].progress_apply(lambda x:x/(max1-min1))


{cols}...: 100%|██████████| 3735387/3735387 [00:04<00:00, 750770.62it/s]

{cols}...: 100%|██████████| 3735387/3735387 [00:04<00:00, 765160.55it/s]

{cols}...:  32%|███▏      | 1193511/3735387 [00:01<44:29, 952.09it/s]  


{cols}...: 100%|██████████| 3735387/3735387 [00:04<00:00, 749418.69it/s]

{cols}...: 100%|██████████| 3735387/3735387 [00:05<00:00, 735408.71it/s]

{cols}...:  67%|██████▋   | 2518446/3735387 [00:03<00:06, 197128.24it/s]


{cols}...: 100%|██████████| 3735387/3735387 [00:04<00:00, 780241.28it/s]

{cols}...: 100%|██████████| 3735387/3735387 [00:05<00:00, 741762.00it/s]

{cols}...: 100%|██████████| 3735387/3735387 [00:05<00:00, 742780.67it/s]


In [21]:
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,qid,qid_enc,...,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,uid,uid_enc,wk
0,3865,-0.122170,3.0,2,22,0.021705,0.0,0.0,Q2166419046,0.302068,...,0,0,0.825391,0.655172,0.611241,0.596141,1.0,M401693808,0.780729,1
1,3862,-0.015598,3.0,2,15,-0.182562,0.0,0.0,Q604029601,0.897803,...,0,0,0.593359,0.789655,0.000000,0.325402,1.0,M2317670257,0.340633,5
2,3867,0.000000,2.0,2,4,0.070942,0.0,32.0,Q2443223942,0.373537,...,0,0,0.215234,0.779310,0.440281,0.524116,1.0,M3544409350,0.658686,3
3,3861,-0.302475,4.0,1,20,-0.191518,0.0,3.0,Q795459266,0.947105,...,0,0,0.593359,0.789655,0.000000,0.325402,1.0,M2818659842,0.470661,4
4,3862,-0.095008,3.0,0,8,-0.176807,1.0,0.0,Q110462128,0.026651,...,0,0,0.844141,0.106897,0.927400,0.924759,1.0,M848334644,0.960845,5


In [22]:
# target编码
logging.info("feature size %s", len(feature_cols))
num_x = len(feature_cols)
X_train_all = data.iloc[:train_label][feature_cols]
y_train_all = data.iloc[:train_label]['label']
# test = data.iloc[len(train_label):]
# # del data
# assert len(test) == sub_size

# logging.info("train shape %s", train_label.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

[2019-12-21 18:55:33,979] INFO in <ipython-input-22-f1315af55f35>: feature size 21


In [23]:
X_train.head()

,follow_topic,freq,gender,hour,inter_topic,q_ans_count,qid_enc,score,u_ans_count,uf_b1,...,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,uid_enc,wk
0,-0.122170,3.0,2,22,0.021705,0.0,0.302068,297,0.0,0,...,0,0,0,0.825391,0.655172,0.611241,0.596141,1.0,0.780729,1
1,-0.015598,3.0,2,15,-0.182562,0.0,0.897803,415,2.0,1,...,0,0,0,0.593359,0.789655,0.000000,0.325402,1.0,0.340633,5
2,0.000000,2.0,2,4,0.070942,32.0,0.373537,296,0.0,1,...,0,0,0,0.215234,0.779310,0.440281,0.524116,1.0,0.658686,3
3,-0.302475,4.0,1,20,-0.191518,3.0,0.947105,380,6.0,1,...,0,0,0,0.593359,0.789655,0.000000,0.325402,1.0,0.470661,4
4,-0.095008,3.0,0,8,-0.176807,0.0,0.026651,719,56.0,1,...,0,0,0,0.844141,0.106897,0.927400,0.924759,1.0,0.960845,5


In [24]:
X_train_array = np.array(X_train)
y_train_array = np.array(y_train)
X_val_array = np.array(X_val)
y_val_array = np.array(y_val)

In [25]:
y_train_array=y_train_array.reshape(-1,1)
y_val_array=y_val_array.reshape(-1,1)

In [26]:
print(y_train_array)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [1.]]


In [27]:
enc=OneHotEncoder()
enc.fit([[0],[1]])
y_train_x=enc.transform(y_train_array).toarray()
y_val_x=enc.transform(y_val_array).toarray()

print(y_train_x[0])

[1. 0.]


In [28]:
model = kk.models.Sequential()
model.add(Dense(25,input_dim=num_x,activation='relu'))
# model.add(kk.layers.Dropout(0.2))
# model.add(Dense(40,activation='relu'))
# model.add(kk.layers.Dropout(0.2))
# model.add(Dense(25,activation='relu'))
# model.add(kk.layers.Dropout(0.2))
model.add(Dense(40,activation='relu'))
model.add(kk.layers.Dropout(0.2))
model.add(Dense(25,activation='relu'))
model.add(kk.layers.Dropout(0.2))
# model.add(Dense(20,activation='relu'))
# model.add(kk.layers.Dropout(0.2))
model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train_array,y_train_x,epochs=100,batch_size=200)


Epoch 1/100
2074934/2074934 [==============================] - 20s - loss: 0.4408 - acc: 0.8389    
Epoch 2/100
2074934/2074934 [==============================] - 20s - loss: 0.4140 - acc: 0.8399    
Epoch 3/100
2074934/2074934 [==============================] - 20s - loss: 0.4111 - acc: 0.8399    
Epoch 4/100
2074934/2074934 [==============================] - 19s - loss: 0.4090 - acc: 0.8401    
Epoch 5/100
2074934/2074934 [==============================] - 20s - loss: 0.4067 - acc: 0.8404    
Epoch 6/100
2074934/2074934 [==============================] - 20s - loss: 0.4050 - acc: 0.8403    
Epoch 7/100
2074934/2074934 [==============================] - 20s - loss: 0.4041 - acc: 0.8402    
Epoch 8/100
2074934/2074934 [==============================] - 20s - loss: 0.4034 - acc: 0.8402    
Epoch 9/100
2074934/2074934 [==============================] - 20s - loss: 0.4031 - acc: 0.8403    
Epoch 10/100
2074934/2074934 [==============================] - 20s - loss: 0.4027 - acc: 0.8403    

2074934/2074934 [==============================] - 20s - loss: 0.3997 - acc: 0.8407    
Epoch 83/100
2074934/2074934 [==============================] - 20s - loss: 0.3995 - acc: 0.8409    
Epoch 84/100
2074934/2074934 [==============================] - 20s - loss: 0.3996 - acc: 0.8407    
Epoch 85/100
2074934/2074934 [==============================] - 20s - loss: 0.3995 - acc: 0.8408    
Epoch 86/100
2074934/2074934 [==============================] - 20s - loss: 0.3995 - acc: 0.8408    
Epoch 87/100
2074934/2074934 [==============================] - 20s - loss: 0.3996 - acc: 0.8408    
Epoch 88/100
2074934/2074934 [==============================] - 20s - loss: 0.3997 - acc: 0.8406    
Epoch 89/100
2074934/2074934 [==============================] - 20s - loss: 0.3996 - acc: 0.8407    
Epoch 90/100
2074934/2074934 [==============================] - 20s - loss: 0.3995 - acc: 0.8407    
Epoch 91/100
2074934/2074934 [==============================] - 20s - loss: 0.3995 - acc: 0.8407    
Epo

In [30]:
scores = model.evaluate(x=X_val_array,y=y_val_x)
print('\n%s :%.5f' %(model.metrics_names[1],scores[1]))

516448/518735 [============================>.] - ETA: 0s
acc :0.84002
